In [55]:
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
from langgraph.graph import StateGraph, START, END
from typing import TypedDict

# .env 파일에서 환경 변수 로드
load_dotenv()

True

In [56]:
#### GPT-4o-mini 설정
gpt4o_mini = ChatOpenAI(
    model_name="gpt-4o-mini",  # GPT-4o-mini에 해당하는 모델명
    temperature=0.7,
    max_tokens=150,
)

#### GPT-4o 설정
gpt4o = ChatOpenAI(
    model_name="gpt-4o",  # GPT-4o에 해당하는 모델명
    temperature=0.7,
    max_tokens=300,
)

In [4]:
#### GPT-4o-mini 사용 예시
response_mini = gpt4o_mini.invoke([HumanMessage(content="안녕 넌 어때?")])
print(response_mini.content, '\n\n')

안녕하세요! 저는 잘 지내고 있어요. 당신은 어떠신가요? 도움이 필요하시면 언제든지 말씀해 주세요! 




In [53]:
#### 그래프
class MyState(TypedDict): # 그래프의 상태를 정의하는 클래스
    user_question: str
    counter: int
    answer: str

graph = StateGraph(MyState) # StateGraph 인스턴스 생성


#### 노드
def cnt_p_1(state): # 카운터 1 증가
    
    return {"counter": state["counter"] + 1}


def cnt_m_1(state): # 카운터 1 증가
    
    return {"counter": state["counter"] - 1}


def question_to_model(state):
    response_mini = 500
    
    return {"answer": response_mini}


def check_question(state):
    print('condition_function :', check_question, state["answer"])
    if state["answer"] < 50:
        print(50)
        return 50
    elif state["answer"] < 100: 
        print(100)
        return 100
    elif state["answer"] < 200:
        print(200)
        return 200
    else:
        print(999)
        return 999
        


graph.add_node("cnt_p_1", cnt_p_1)
graph.add_node("cnt_m_1", cnt_m_1)
graph.add_node("question_to_model", question_to_model)
graph.add_node("check_question", check_question)


#### 엣지
graph.add_edge(START, "question_to_model")

graph.add_conditional_edges(
                "question_to_model",
                check_question,
                {
                    50: "cnt_p_1",
                    100: "cnt_m_1",
                    200: "cnt_m_1",
                    999: END
                })


In [62]:
#### 그래프 컴파일
app = graph.compile()



#### 그래프 실행
user_question = '코드 프로그래머스 웹 사이트에서 sql과 관련된 문제 하나만 가져와줄래?'
counter = 0
answer = ''


result = app.invoke({"user_question": user_question, "counter": counter, "answer": answer})
print(result)

{'user_question': '코드 프로그래머스 웹 사이트에서 sql과 관련된 문제 하나만 가져와줄래?', 'counter': 0, 'answer': AIMessage(content='프로그래머스 웹사이트에서 SQL 관련 문제를 직접 가져올 수는 없지만, SQL 문제의 예시를 제공해드릴 수 있습니다. 아래는 SQL 문제의 예시입니다.\n\n### 문제: 직원의 급여 정보\n\n**설명:**\n`Employees`라는 테이블이 있습니다. 이 테이블은 다음과 같은 열을 가집니다:\n\n- `id` (INT): 직원의 고유 ID\n- `name` (VARCHAR): 직원의 이름\n- `salary` (INT): 직원의 급여\n- `department_id` (INT): 직원이 속한 부서의 ID\n\n부서별로 평균 급여를 구하는 SQL 쿼리를 작성하세요. 결과', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 150, 'prompt_tokens': 28, 'total_tokens': 178, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'length', 'logprobs': None}, id='run-76180fda-798b-44fe-8cae-90f6eeedf6e8-0')}


In [ ]:
from graphviz import Digraph

# 상태 그래프 정의
dot = Digraph()

# 노드 추가
dot.node("START", "START", shape="circle")
dot.node("increment", "Increment Counter", shape="box")
dot.node("END", "END", shape="doublecircle")

# 엣지 추가
dot.edge("START", "increment")
dot.edge("increment", "END")

# 그래프 출력
dot.render("state_graph", format="png", cleanup=True)  # PNG 파일로 저장
dot.view()  # 디폴트 뷰어에서 열기

In [8]:
def stategraph_to_graphviz(stategraph, start_label="START", end_label="END"):
    """
    StateGraph 객체를 Graphviz 형식으로 변환.
    
    Parameters:
        stategraph: StateGraph 객체 (가상의 데이터 구조 사용).
        start_label (str): 시작 노드의 라벨.
        end_label (str): 종료 노드의 라벨.
    Returns:
        Digraph: Graphviz의 Digraph 객체.
    """
    dot = Digraph()
    
    # 상태 그래프에서 노드와 엣지 추출
    nodes = stategraph.get_nodes()  # 노드 리스트 가져오기
    edges = stategraph.get_edges()  # 엣지 리스트 가져오기

    # 노드 추가
    for node in nodes:
        shape = "circle" if node in (start_label, end_label) else "box"
        dot.node(node, node, shape=shape)
    
    # 엣지 추가
    for src, dst in edges:
        dot.edge(src, dst)

    return dot